
# Space

In [ ]:
import os
import sys 
import logging
import pandas as pd 
from pprint import pprint 

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
pprint(SPACE)

# Available Packages
import pandas as pd
from datetime import datetime 

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


# Trigger Events

In [ ]:
import datasets
from recfldtkn.loadtools import load_ds_rec_and_info
from recfldtkn.configfn import load_cohort_args, load_record_args

base_config = load_cohort_args(recfldtkn_config_path, SPACE)
print(base_config)

# TriggerCaseMethod

In [ ]:
######################################
TriggerCaseMethod = 'CGM5MinEntry'

cohort_label_list = [1]
######################################

In [ ]:
from recfldtkn.loadtools import load_module_variables

pypath = os.path.join(base_config['trigger_pyfolder'], f'{TriggerCaseMethod}.py')
module = load_module_variables(pypath)
TriggerRecName = module.TriggerRecName
case_id_columns = module.case_id_columns
special_columns = module.special_columns
convert_TriggerEvent_to_Caseset = module.convert_TriggerEvent_to_Caseset

ds_rec, _ = load_ds_rec_and_info(TriggerRecName, base_config, cohort_label_list)
df_case = convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, base_config)
df_case

In [ ]:
#####################
df_case = df_case.sample(1000, random_state=42)
#####################
df_case

# Case from Trigger to Learning

## Tag: TagPttBasicInfo

In [ ]:
##################################
TagMethod = 'TagWDPttBasicInfo'
##################################

import inspect

InfoRecName = 'P'
subgroup_columns = ['Gender', 'YearOfBirth', 'DiseaseType']

def fn_case_tagging(df_case, ds_info, subgroup_columns, base_config):
    RootID = base_config['RootID']
    ds_info = ds_info.select_columns([RootID] + subgroup_columns)
    df_info = ds_info.to_pandas()
    df_info['cohort'] = df_info[RootID].apply(lambda x: 'C' + str(x)[:-base_config['RootIDLength']])
    # df_info.head()
    final_columns = [RootID] + subgroup_columns + ['cohort']

    df_case = pd.merge(df_case, df_info[final_columns], how = 'left')
    df_case['Age'] = df_case['ObsDT'].dt.year - df_case['YearOfBirth']
    return df_case

fn_case_tagging.fn_string = inspect.getsource(fn_case_tagging)

In [ ]:
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

prefix = ['import pandas as pd', 'import numpy as np']
string_variables = [InfoRecName]
iterative_variables = [subgroup_columns]
fn_variables = [fn_case_tagging]
pycode = convert_variables_to_pystirng(string_variables = string_variables, 
                                       iterative_variables = iterative_variables, 
                                       fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{TagMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

In [ ]:
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{TagMethod}.py')
module = load_module_variables(pypath)
InfoRecName = module.InfoRecName
subgroup_columns = module.subgroup_columns
fn_case_tagging = module.fn_case_tagging

In [ ]:
ds_info, _ = load_ds_rec_and_info(InfoRecName, base_config, cohort_label_list)
df_case = fn_case_tagging(df_case, ds_info, subgroup_columns, base_config)
df_case

In [ ]:
series_time = df_case['ObsDT']
print(series_time.min(), series_time.max())

In [ ]:
df_case['Age'].fillna('Missing').value_counts().sort_index()

In [ ]:
df_case['Age'].value_counts().sort_index().plot()#.reset_index()#.plot()

## Filter: FilterBasicPttWD

In [ ]:
##################################
FilterMethod = 'FilterBasicPttWD'
##################################
import inspect

def fn_case_filtering(df_case):
    df_case = df_case[df_case['Age'] >= 18].reset_index(drop = True)
    return df_case

fn_case_filtering.fn_string = inspect.getsource(fn_case_filtering)

In [ ]:
from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_case_filtering]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

In [ ]:
# Trigger2LearningCaseMethod
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
module = load_module_variables(pypath)
fn_case_filtering = module.fn_case_filtering

In [ ]:
print(df_case.shape)
df_case = fn_case_filtering(df_case)
print(df_case.shape)

In [ ]:
df_case['Age'].value_counts().sort_index().plot()#.reset_index()#.plot()

## CaseFeat: CFQuery_Bf24hCGMrn

In [ ]:
##################################
CaseFeatQueryMethod = 'CFQuery_Bf24hCGMrn'
##################################
import inspect

def fn_casefeat_querying(df_case, base_config):
    from recfldtkn.observer import get_fn_case_GammaFullInfo
    import datasets
    #############################
    Gamma_Config = {
        'case_observations': [
            'Bf24hCGMrn:ro.CGM5Min-Bf24H_ct.RecNum', # CO
        ],
        'name_CaseGamma': 'InputCatCOs', # CF
    }
    #############################

    RecName_to_dsRec = {}
    RecName_to_dsRecInfo = {}
    use_CF_from_disk = False 
    use_CO_from_disk = True
    CaseFeatInfo = get_fn_case_GammaFullInfo(Gamma_Config, 
                                                base_config, 
                                                RecName_to_dsRec, 
                                                RecName_to_dsRecInfo, 
                                                df_case,
                                                use_CF_from_disk,
                                                use_CO_from_disk)
    
    ds_case = datasets.Dataset.from_pandas(df_case)

    FnCaseFeatGamma = CaseFeatInfo['FnCaseFeatGamma']
    batch_size = CaseFeatInfo.get('batch_size', 1000)
    CaseFeatName = CaseFeatInfo['CaseFeatName']
    CF_vocab = CaseFeatInfo['CF_vocab']

    # start = datetime.now()
    ds_case = ds_case.map(FnCaseFeatGamma, 
                            batched = True, 
                            batch_size= batch_size, 
                            load_from_cache_file = False, 
                            new_fingerprint = CaseFeatName)
    #end = datetime.now()
    # print('Elipse Time: ', end - start)
    # print(ds_case)

        
    rename_dict = {i: CaseFeatName + ':' + i for i in CF_vocab}
    df_case = ds_case.to_pandas().rename(columns=rename_dict)
    return df_case, CaseFeatInfo

fn_casefeat_querying.fn_string = inspect.getsource(fn_casefeat_querying)


from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_casefeat_querying]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{CaseFeatQueryMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)


# Trigger2LearningCaseMethod
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{CaseFeatQueryMethod}.py')
module = load_module_variables(pypath)
fn_casefeat_querying = module.fn_casefeat_querying


In [ ]:
df_case, CaseFeatInfo = fn_casefeat_querying(df_case, base_config)
df_case

## TagCF: TagCaseFeat_Bf24hCGMrn

In [ ]:
##################################
TagCaseFeatMethod = 'TagCaseFeat_Bf24hCGMrn'
##################################

import inspect


def fn_case_tagging_on_casefeat(df_case, CaseFeatInfo):
    CF_vocab = CaseFeatInfo['CF_vocab']
    CaseFeatName = CaseFeatInfo['CaseFeatName']
    input_ids_name = CaseFeatName + ':' + 'input_ids'
    input_wgts_name = CaseFeatName + ':' + 'input_wgts'

    tkn_name = 'Bf24hCGMrn:recnum'
    tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    df_case['BfCGMrn'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)

    # tkn_name = 'Af2hCGMrn:recnum'
    # tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    # df_case['AfCGMrn'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)


    return df_case

fn_case_tagging_on_casefeat.fn_string = inspect.getsource(fn_case_tagging_on_casefeat)

In [ ]:
from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_case_tagging_on_casefeat] # <----------------
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{TagCaseFeatMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)


# Trigger2LearningCaseMethod
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{TagCaseFeatMethod}.py')
module = load_module_variables(pypath)
fn_case_tagging_on_casefeat = module.fn_case_tagging_on_casefeat

In [ ]:
df_case = fn_case_tagging_on_casefeat(df_case, CaseFeatInfo)
df_case.head()

## CaseFeat: CFQuery_Af2hCGM

In [ ]:
##################################
CaseFeatQueryMethod = 'CFQuery_Af2hCGMrn'
##################################
import inspect

def fn_casefeat_querying(df_case, base_config):
    from recfldtkn.observer import get_fn_case_GammaFullInfo
    import datasets
    #############################
    Gamma_Config = {
        'case_observations': [
            'Af2hCGMrn:ro.CGM5Min-Af2H_ct.RecNum', # CO
        ],
        'name_CaseGamma': 'InputCatCOs', # CF
    }
    #############################

    RecName_to_dsRec = {}
    RecName_to_dsRecInfo = {}
    use_CF_from_disk = False 
    use_CO_from_disk = True
    CaseFeatInfo = get_fn_case_GammaFullInfo(Gamma_Config, 
                                                base_config, 
                                                RecName_to_dsRec, 
                                                RecName_to_dsRecInfo, 
                                                df_case,
                                                use_CF_from_disk,
                                                use_CO_from_disk)
    
    ds_case = datasets.Dataset.from_pandas(df_case)

    FnCaseFeatGamma = CaseFeatInfo['FnCaseFeatGamma']
    batch_size = CaseFeatInfo.get('batch_size', 1000)
    CaseFeatName = CaseFeatInfo['CaseFeatName']
    CF_vocab = CaseFeatInfo['CF_vocab']

    # start = datetime.now()
    ds_case = ds_case.map(FnCaseFeatGamma, 
                            batched = True, 
                            batch_size= batch_size, 
                            load_from_cache_file = False, 
                            new_fingerprint = CaseFeatName)
    # end = datetime.now()
    # print('Elipse Time: ', end - start)
    # print(ds_case)

        
    rename_dict = {i: CaseFeatName + ':' + i for i in CF_vocab}
    df_case = ds_case.to_pandas().rename(columns=rename_dict)
    return df_case, CaseFeatInfo

fn_casefeat_querying.fn_string = inspect.getsource(fn_casefeat_querying)


from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_casefeat_querying]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{CaseFeatQueryMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)


# Trigger2LearningCaseMethod
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{CaseFeatQueryMethod}.py')
module = load_module_variables(pypath)
fn_casefeat_querying = module.fn_casefeat_querying


In [ ]:
df_case, CaseFeatInfo = fn_casefeat_querying(df_case, base_config)
df_case

## TagCF: TagCaseFeat_Af2hCGMrn

In [ ]:
##################################
TagCaseFeatMethod = 'TagCaseFeat_Af2hCGMrn'
##################################

import inspect


def fn_case_tagging_on_casefeat(df_case, CaseFeatInfo):
    CF_vocab = CaseFeatInfo['CF_vocab']
    CaseFeatName = CaseFeatInfo['CaseFeatName']
    input_ids_name = CaseFeatName + ':' + 'input_ids'
    input_wgts_name = CaseFeatName + ':' + 'input_wgts'

    # tkn_name = 'Bf24hCGMrn:recnum'
    # tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    # df_case['BfCGMrn'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)

    tkn_name = 'Af2hCGMrn:recnum'
    tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    df_case['AfCGMrn'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)

    return df_case

fn_case_tagging_on_casefeat.fn_string = inspect.getsource(fn_case_tagging_on_casefeat)

In [ ]:
from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_case_tagging_on_casefeat] # <----------------
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{TagCaseFeatMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)



# Trigger2LearningCaseMethod
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{TagCaseFeatMethod}.py')
module = load_module_variables(pypath)
fn_case_tagging_on_casefeat = module.fn_case_tagging_on_casefeat

In [ ]:
df_case = fn_case_tagging_on_casefeat(df_case, CaseFeatInfo)
df_case.head()

## Filter: BfCGMgeq280

In [ ]:
##################################
FilterMethod = 'FilterBfCGMgeq280'
##################################
import inspect

def fn_case_filtering(df_case):
    df_case = df_case[df_case['BfCGMrn'] >= 280].reset_index(drop = True)
    return df_case

fn_case_filtering.fn_string = inspect.getsource(fn_case_filtering)

In [ ]:
from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_case_filtering]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

In [ ]:
# Trigger2LearningCaseMethod
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
module = load_module_variables(pypath)
fn_case_filtering = module.fn_case_filtering

In [ ]:
print(df_case.shape)
df_case = fn_case_filtering(df_case)
print(df_case.shape)

## Filter: AfCGMgeq24

In [ ]:
##################################
FilterMethod = 'FilterAfCGMgeq24'
##################################
import inspect

def fn_case_filtering(df_case):
    df_case = df_case[df_case['AfCGMrn'] >= 24].reset_index(drop = True)
    return df_case

fn_case_filtering.fn_string = inspect.getsource(fn_case_filtering)

In [ ]:
from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_case_filtering]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

In [ ]:
# Trigger2LearningCaseMethod
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
module = load_module_variables(pypath)
fn_case_filtering = module.fn_case_filtering

In [ ]:
print(df_case.shape)
df_case = fn_case_filtering(df_case)
print(df_case.shape)

## CaseFeat: QBf1mMEDAL

In [ ]:
# Gamma_Config = {
#     'case_observations': [
#         'Bf1mFoodRec:ro.FoodRec-Bf1M_ct.RecNum', # CO
#         # 'Af2hCGMrn:ro.CGM5Min-Af2H_ct.RecNum',
#     ],
#     'name_CaseGamma': 'InputCatCOs', # CF
# }

In [ ]:
##################################
CaseFeatQueryMethod = 'CFQuery_Bf1mMEDALrn'
##################################
import inspect

def fn_casefeat_querying(df_case, base_config):
    from recfldtkn.observer import get_fn_case_GammaFullInfo
    import datasets
    #############################
    Gamma_Config = {
        'case_observations': [
            # Med
            'Bf1mMedPresrn:ro.MedPres-Bf1M_ct.RecNum',
            'Bf1mMedAdmin:ro.MedAdmin-Bf1M_ct.RecNum',
            'Bf1mImpMed:ro.ImpMed-Bf1M_ct.RecNum',

            # Edu
            'Bf1mCmtrn:ro.Cmt-Bf1M_ct.RecNum',
            'Bf1mLessonProgrn:ro.LessonProg-Bf1M_ct.RecNum',

            # Diet
            'Bf1mFoodRecrn:ro.FoodRec-Bf1M_ct.RecNum', 
            'Bf1mCarbrn:ro.Carb-Bf1M_ct.RecNum',

            # Activity
            'Bf1mExerciseRecrn:ro.Exercise-Bf1M_ct.RecNum',
            'Bf1mSleepRecrn:ro.Sleep-Bf1M_ct.RecNum',
            'Bf1mStepRecrn:ro.Step-Bf1M_ct.RecNum',

            # LabVital
            'Bf1mBPrn:ro.BP-Bf1M_ct.RecNum',
            'Bf1mWeighUrn:ro.WeightU-Bf1M_ct.RecNum',
            'Bf1mLabTestRestrn:ro.LabTestResult-Bf1M_ct.RecNum',

            # CO
        ],
        'name_CaseGamma': 'InputCatCOs', # CF
    }
    #############################

    RecName_to_dsRec = {}
    RecName_to_dsRecInfo = {}
    use_CF_from_disk = False 
    use_CO_from_disk = True
    CaseFeatInfo = get_fn_case_GammaFullInfo(Gamma_Config, 
                                                base_config, 
                                                RecName_to_dsRec, 
                                                RecName_to_dsRecInfo, 
                                                df_case,
                                                use_CF_from_disk,
                                                use_CO_from_disk)
    
    ds_case = datasets.Dataset.from_pandas(df_case)

    FnCaseFeatGamma = CaseFeatInfo['FnCaseFeatGamma']
    batch_size = CaseFeatInfo.get('batch_size', 1000)
    CaseFeatName = CaseFeatInfo['CaseFeatName']
    CF_vocab = CaseFeatInfo['CF_vocab']

    # start = datetime.now()
    ds_case = ds_case.map(FnCaseFeatGamma, 
                            batched = True, 
                            batch_size= batch_size, 
                            load_from_cache_file = False, 
                            new_fingerprint = CaseFeatName)
    # end = datetime.now()
    # print('Elipse Time: ', end - start)
    # print(ds_case)

        
    rename_dict = {i: CaseFeatName + ':' + i for i in CF_vocab}
    df_case = ds_case.to_pandas().rename(columns=rename_dict)
    return df_case, CaseFeatInfo

fn_casefeat_querying.fn_string = inspect.getsource(fn_casefeat_querying)


# from recfldtkn.loadtools import convert_variables_to_pystirng

# prefix = ['import pandas as pd', 'import numpy as np']
# fn_variables = [fn_casefeat_querying]
# pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
# pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{CaseFeatQueryMethod}.py')
# print(pypath)
# if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
# with open(pypath, 'w') as file: file.write(pycode)


# # Trigger2LearningCaseMethod
# pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{CaseFeatQueryMethod}.py')
# module = load_module_variables(pypath)
# fn_casefeat_querying = module.fn_casefeat_querying


In [ ]:
from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_casefeat_querying]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{CaseFeatQueryMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)


# Trigger2LearningCaseMethod
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{CaseFeatQueryMethod}.py')
module = load_module_variables(pypath)
fn_casefeat_querying = module.fn_casefeat_querying


In [ ]:
df_case, CaseFeatInfo = fn_casefeat_querying(df_case, base_config)
df_case

In [ ]:
CaseFeatInfo['CaseFeatName']

In [ ]:
CaseFeatInfo['CF_vocab']

## TagCF: TagCaseFeat_Bf1mMEDALrn

In [ ]:
##################################
TagCaseFeatMethod = 'TagCaseFeat_Bf1mMEDALrn'
##################################

import inspect


def fn_case_tagging_on_casefeat(df_case, CaseFeatInfo):
    CF_vocab = CaseFeatInfo['CF_vocab']
    CaseFeatName = CaseFeatInfo['CaseFeatName']
    input_ids_name = CaseFeatName + ':' + 'input_ids'
    input_wgts_name = CaseFeatName + ':' + 'input_wgts'

    tkn_name = 'Bf1mMedPresrn:recnum'
    tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    df_case['BfMedPresRN'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)

    tkn_name = 'Bf1mMedAdmin:recnum'
    tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    df_case['BfMedAdminRN'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)

    tkn_name = 'Bf1mImpMed:recnum'
    tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    df_case['BfImpMedRN'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)

    tkn_name = 'Bf1mCmtrn:recnum'
    tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    df_case['BfCmtRN'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)

    tkn_name = 'Bf1mLessonProgrn:recnum'
    tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    df_case['BfLessonProgRN'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)

    tkn_name = 'Bf1mFoodRecrn:recnum'
    tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    df_case['BfFoodRecRN'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)
    
    tkn_name = 'Bf1mCarbrn:recnum'
    tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    df_case['BfCarbRN'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)

    tkn_name = 'Bf1mExerciseRecrn:recnum'
    tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    df_case['BfExerciseRecRN'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)

    tkn_name = 'Bf1mSleepRecrn:recnum'
    tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    df_case['BfSleepRecRN'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)

    tkn_name = 'Bf1mStepRecrn:recnum'
    tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    df_case['BfStepRecRN'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)

    tkn_name = 'Bf1mBPrn:recnum'
    tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    df_case['BfBPRN'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)

    tkn_name = 'Bf1mWeighUrn:recnum'
    tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    df_case['BfWeighURN'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)

    tkn_name = 'Bf1mLabTestRestrn:recnum'
    tkn_id = CF_vocab['input_ids']['tkn2tid'][tkn_name]
    df_case['BfLabTestRestRN'] = df_case.apply(lambda x: x[input_wgts_name][list(x[input_ids_name]).index(tkn_id)], axis = 1)


    return df_case

fn_case_tagging_on_casefeat.fn_string = inspect.getsource(fn_case_tagging_on_casefeat)

In [ ]:
# from recfldtkn.loadtools import convert_variables_to_pystirng

# prefix = ['import pandas as pd', 'import numpy as np']
# fn_variables = [fn_case_tagging_on_casefeat] # <----------------
# pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
# pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{TagCaseFeatMethod}.py')
# print(pypath)
# if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
# with open(pypath, 'w') as file: file.write(pycode)



# # Trigger2LearningCaseMethod
# pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{TagCaseFeatMethod}.py')
# module = load_module_variables(pypath)
# fn_case_tagging_on_casefeat = module.fn_case_tagging_on_casefeat

In [ ]:
df_case = fn_case_tagging_on_casefeat(df_case, CaseFeatInfo)
df_case.head()